In [17]:
from pathlib import Path
import os

repo_url = "https://github.com/tallesmedeiros/DecisionMaking.git"
repo_dir = Path.cwd()

# Verifica se o repositório já está disponível ou se precisa ser clonado
if (repo_dir / 'plan_generator.py').exists():
    target_dir = repo_dir
    print('✅ Repositório já disponível neste diretório.')
elif (repo_dir / 'DecisionMaking').exists():
    target_dir = repo_dir / 'DecisionMaking'
    print('ℹ️ Pasta DecisionMaking já existe. Entrando nela.')
else:
    target_dir = repo_dir / 'DecisionMaking'
    print('⬇️ Clonando repositório com os arquivos necessários...')
    !git clone {repo_url}

os.chdir(target_dir)
print('📁 Diretório atual:', Path.cwd())
!ls -la *.py


Cloning into 'DecisionMaking'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 70 (delta 22), reused 45 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 121.34 KiB | 12.13 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/DecisionMaking/DecisionMaking
-rw-r--r-- 1 root root  9465 Nov 23 22:43 cli.py
-rw-r--r-- 1 root root  2085 Nov 23 22:43 example_with_zones.py
-rw-r--r-- 1 root root 14605 Nov 23 22:43 intervals_integration.py
-rw-r--r-- 1 root root 16827 Nov 23 22:43 notebook_widgets.py
-rw-r--r-- 1 root root 16937 Nov 23 22:43 pdf_export.py
-rw-r--r-- 1 root root 47125 Nov 23 22:43 plan_generator.py
-rw-r--r-- 1 root root  6359 Nov 23 22:43 plot_utils.py
-rw-r--r-- 1 root root 19038 Nov 23 22:43 running_plan.py
-rw-r--r-- 1 root root  7335 Nov 23 22:43 test_enhanced.py
-rw-r--r-- 1 root root  3026 Nov 23 22:43 test_example.py
-rw-r--r-- 1 root root  6340

# 🏃‍♂️ Criador de Plano de Treino Personalizado - Interativo

## Bem-vindo ao seu assistente de treino inteligente!

Este notebook usa **widgets interativos** para criar seu plano de treino personalizado.

**✨ Novidades:**
- 🎨 Interface visual com sliders, dropdowns e campos de texto
- 📊 Visualizações automáticas com gráficos
- 📄 Exportação em PDF com todo o plano
- 🚀 Mais fácil de usar - sem precisar entender código!

---

## 📦 Instalação de Dependências (Google Colab)

Se você estiver usando o Google Colab, execute a célula abaixo primeiro:

In [18]:
# Instalar dependências necessárias (apenas no Google Colab)
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Detectado Google Colab")
    print("📦 Instalando dependências...")
    !pip install -q ipywidgets reportlab matplotlib
    print("✅ Dependências instaladas!")
except:
    IN_COLAB = False
    print("💻 Ambiente local detectado")

🌐 Detectado Google Colab
📦 Instalando dependências...
✅ Dependências instaladas!


---

## 🎯 Escolha o Modo de Criação

Temos dois modos disponíveis:

1. **🚀 Modo Simples**: Criação rápida com informações básicas
2. **⚙️ Modo Completo**: Personalização total com zonas de treino, lesões, etc.

Execute uma das células abaixo de acordo com sua preferência:

### ⚙️ MODO COMPLETO - Personalização Total

Execute esta célula para criar um plano totalmente personalizado:

In [26]:
from notebook_widgets import create_complete_plan_widgets

# Criar interface completa
widgets_completo = create_complete_plan_widgets()

Text(value='João Silva', description='Nome:', layout=Layout(width='400px'), style=DescriptionStyle(description…

IntSlider(value=30, description='Idade:', layout=Layout(width='400px'), max=80, min=15, style=SliderStyle(desc…

FloatSlider(value=70.0, description='Peso (kg):', layout=Layout(width='400px'), max=150.0, min=40.0, step=0.5,…

IntSlider(value=175, description='Altura (cm):', layout=Layout(width='400px'), max=220, min=140, style=SliderS…

Dropdown(description='Sexo:', layout=Layout(width='400px'), options=(('Masculino', 'M'), ('Feminino', 'F'), ('…

...

Dropdown(description='Método de cálculo:', layout=Layout(width='400px'), options=(('Jack Daniels (recomendado)…

SelectMultiple(description='Lesões atuais:', layout=Layout(height='120px', width='400px'), options=('Fascite P…

SelectMultiple(description='Lesões prévias:', layout=Layout(height='120px', width='400px'), options=('Fascite …

Button(button_style='success', description='🚀 Gerar Plano Personalizado', layout=Layout(height='50px', width='…

Output()

---

## 📊 Visualizar Plano Gerado

Após gerar o plano usando os widgets acima, execute as células abaixo para diferentes visualizações:

### 📋 Resumo Visual do Plano

In [27]:
# Pegar o plano gerado
try:
    if 'widgets_completo' in locals() and widgets_completo and widgets_completo.plan:
        plan = widgets_completo.plan
    elif 'widgets_simples' in locals() and widgets_simples and widgets_simples.plan:
        plan = widgets_simples.plan
    else:
        print("⚠️  Nenhum plano gerado ainda. Execute uma das células acima primeiro!")
        plan = None

    if plan:
        print("\n" + "="*70)
        print("📋 RESUMO DO SEU PLANO DE TREINO")
        print("="*70)
        plan.print_visual()
except Exception as e:
    print(f"❌ Erro: {e}")


📋 RESUMO DO SEU PLANO DE TREINO

🏃‍♂️ PLANO DE TREINO: Plano 5K - João Silva
🎯 Meta: 5K
📊 Nível: Advanced
📅 Duração: 3 semanas
🗓️  Dias de treino: 4 dias/semana
🚀 Início: 24/11/2025 (Monday)
🏁 Prova: 15/12/2025 (Monday)
📏 Kilometragem total: 115.0km

📅 SEMANA 1 (24/11 a 30/11) | 📏 55km total
💡 Welcome to your training plan! Start easy and focus on consistency.
  📍 Monday (24/11): 😴 Descanso
  📍 Tuesday (25/11): 🟢 Fácil: 15km @ 4:21/km [1:00:00]
  📍 Wednesday (26/11): 😴 Descanso
  📍 Thursday (27/11): 🟢 Fácil: 10km @ 4:21/km [30:00]
  📍 Friday (28/11): 🟢 Fácil: 10km @ 4:21/km [30:00]
  📍 Saturday (29/11): 😴 Descanso
  📍 Sunday (30/11): 🟢 Longão: 20km @ 4:44/km [1:30:00]


📅 SEMANA 2 (01/12 a 07/12) | 📏 40km total
💡 Taper week - reduce volume to arrive fresh for race day.
  📍 Monday (01/12): 😴 Descanso
  📍 Tuesday (02/12): 🟢 Fácil: 10km @ 4:21/km [30:00]
  📍 Wednesday (03/12): 😴 Descanso
  📍 Thursday (04/12): 🟢 Fácil: 10km @ 4:21/km [30:00]
  📍 Friday (05/12): 🟢 Fácil: 5km @ 4:21/km [30:

### 📅 Ver Todas as Semanas Detalhadas

In [28]:
if plan:
    plan.print_visual(show_all_weeks=True)
else:
    print("⚠️  Gere um plano primeiro!")


🏃‍♂️ PLANO DE TREINO: Plano 5K - João Silva
🎯 Meta: 5K
📊 Nível: Advanced
📅 Duração: 3 semanas
🗓️  Dias de treino: 4 dias/semana
🚀 Início: 24/11/2025 (Monday)
🏁 Prova: 15/12/2025 (Monday)
📏 Kilometragem total: 115.0km

📅 SEMANA 1 (24/11 a 30/11) | 📏 55km total
💡 Welcome to your training plan! Start easy and focus on consistency.
  📍 Monday (24/11): 😴 Descanso
  📍 Tuesday (25/11): 🟢 Fácil: 15km @ 4:21/km [1:00:00]
  📍 Wednesday (26/11): 😴 Descanso
  📍 Thursday (27/11): 🟢 Fácil: 10km @ 4:21/km [30:00]
  📍 Friday (28/11): 🟢 Fácil: 10km @ 4:21/km [30:00]
  📍 Saturday (29/11): 😴 Descanso
  📍 Sunday (30/11): 🟢 Longão: 20km @ 4:44/km [1:30:00]


📅 SEMANA 2 (01/12 a 07/12) | 📏 40km total
💡 Taper week - reduce volume to arrive fresh for race day.
  📍 Monday (01/12): 😴 Descanso
  📍 Tuesday (02/12): 🟢 Fácil: 10km @ 4:21/km [30:00]
  📍 Wednesday (03/12): 😴 Descanso
  📍 Thursday (04/12): 🟢 Fácil: 10km @ 4:21/km [30:00]
  📍 Friday (05/12): 🟢 Fácil: 5km @ 4:21/km [30:00]
  📍 Saturday (06/12): 😴 Desca

### 📊 Gráficos de Visualização

Veja gráficos bonitos do seu plano:

In [29]:
if plan:
    try:
        from plot_utils import plot_weekly_volume, plot_zone_distribution_stacked, print_zone_summary
        import matplotlib.pyplot as plt

        # Gráfico de volume semanal
        print("📊 Volume Semanal (km)\n")
        fig, ax = plot_weekly_volume(plan)
        plt.show()

        # Gráfico de distribuição de zonas
        if hasattr(plan, 'training_zones') and plan.training_zones:
            print("\n📊 Distribuição de Zonas por Semana\n")
            fig, ax = plot_zone_distribution_stacked(plan)
            plt.show()

            # Resumo textual
            print_zone_summary(plan)
    except ImportError:
        print("⚠️  matplotlib não disponível. Instale com: pip install matplotlib")
    except Exception as e:
        print(f"❌ Erro ao gerar gráficos: {e}")
else:
    print("⚠️  Gere um plano primeiro!")

📊 Volume Semanal (km)



/content/DecisionMaking/plot_utils.py:81: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/usr/local/lib/python3.12/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)


### 📊 Tabela de Zonas de Treino

Veja suas zonas de treino personalizadas (se você informou tempos de prova):

In [30]:
if plan and hasattr(plan, 'training_zones') and plan.training_zones:
    print(plan.training_zones.to_table())
elif plan:
    print("ℹ️  Este plano não tem zonas de treino calculadas.")
    print("   Para ter zonas personalizadas, preencha seus tempos recentes de prova.")
else:
    print("⚠️  Gere um plano primeiro!")

ℹ️  Este plano não tem zonas de treino calculadas.
   Para ter zonas personalizadas, preencha seus tempos recentes de prova.


---

## 💾 Salvar Plano

Salve seu plano em diferentes formatos:

### 📄 Salvar como JSON

In [31]:
if plan:
    # Nome do arquivo baseado no plano
    filename_json = plan.name.replace(" ", "_").replace("/", "_") + ".json"
    plan.save_to_file(filename_json)

    print(f"✅ Plano salvo em: {filename_json}")
    print(f"\n💡 Você pode carregar este plano depois com:")
    print(f"   from running_plan import RunningPlan")
    print(f"   plan = RunningPlan.load_from_file('{filename_json}')")
else:
    print("⚠️  Gere um plano primeiro!")

✅ Plano salvo em: Plano_5K_-_João_Silva.json

💡 Você pode carregar este plano depois com:
   from running_plan import RunningPlan
   plan = RunningPlan.load_from_file('Plano_5K_-_João_Silva.json')


### 📄 Exportar como PDF

**✨ NOVO!** Exporte seu plano completo em PDF com gráficos:

In [32]:
if plan:
    from pdf_export import save_plan_as_pdf

    # Gerar PDF com gráficos
    pdf_file = save_plan_as_pdf(plan, include_graphs=True)

    if pdf_file:
        print(f"\n📥 Para baixar o PDF:")
        if IN_COLAB:
            from google.colab import files
            print(f"   Clique no link abaixo para fazer download:")
            files.download(pdf_file)
        else:
            print(f"   O arquivo está em: {pdf_file}")
else:
    print("⚠️  Gere um plano primeiro!")

📄 Gerando PDF...


/content/DecisionMaking/plot_utils.py:81: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/content/DecisionMaking/pdf_export.py:195: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.savefig(vol_img_path, dpi=150, bbox_inches='tight')


✅ PDF gerado com sucesso: Plano_5K_-_João_Silva.pdf
✅ PDF salvo com sucesso!
📁 Arquivo: Plano_5K_-_João_Silva.pdf
📊 O PDF inclui:
   • Informações do plano
   • Gráficos de volume e distribuição
   • Plano detalhado semana a semana
   • Dicas de treino

💡 Você pode fazer download deste arquivo e imprimir!

📥 Para baixar o PDF:
   Clique no link abaixo para fazer download:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

## 🎉 Parabéns!

Seu plano de treino personalizado está pronto!

### 💡 Próximos Passos:

1. ✅ Baixe o PDF gerado
2. 📱 Imprima ou salve no seu celular
3. 🏃 Siga o plano consistentemente
4. 👂 Escute seu corpo e ajuste quando necessário
5. 🎯 Boa sorte na sua prova!

### 📋 Lembre-se:

- **Consistência** é mais importante que intensidade
- **Descanso** é parte essencial do treino
- **Hidratação** e **nutrição** são fundamentais
- **Ouça seu corpo** - descanse se necessário

---

**🏃‍♂️ Bons treinos e sucesso na sua jornada! 🎉**